In [4]:
import cv2
import numpy as np
from scipy.spatial.distance import cdist

### Code to get HSV Lower and Upper values

[[0, 0, 255], [4, 0, 255]] for road


[[0, 9, 255], [22, 38, 255]] for shop

[[96, 3, 9], [100, 255, 255]] for other building


[[52, 28, 206], [71, 255, 255]] for vegetation

In [3]:
def nothing(x):
    pass
cv2.namedWindow("Trackbars",cv2.WINDOW_NORMAL)



cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)

while(True):
    
    img_og = cv2.imread("Images/tmp_1.png")
    
    hsv = cv2.cvtColor(img_og, cv2.COLOR_BGR2HSV)
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")
    
    
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    res = cv2.bitwise_and(img_og, img_og, mask=mask)
    mask_2 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    stacked = np.hstack((mask_2, img_og, res))
    stacked = cv2.resize(stacked,None,fx=0.4,fy=0.4)
    cv2.imshow('video',stacked)
    #cv2.imshow('og', img_og)
    key = cv2.waitKey(1)
    if key == 27:
        break
      
    if key == ord('s'):
        
        arr1 = [[l_h,l_s,l_v],[u_h, u_s, u_v]]
        print(arr1)
        
        break


cv2.destroyAllWindows()

In [13]:
def findmidpoint(image):
    x_mid, y_mid = int(image.shape[1]/2), int(image.shape[0]/2)

    cv2.circle(image, (x_mid,y_mid), radius=2, color=(0, 0, 255), thickness=-1)
    midpoint = (x_mid,y_mid)
    return midpoint

# Load Upper and Lower Bounds 
road_l_range = np.array([0, 0, 255])
road_u_range = np.array([4, 0, 255])

shop_l_range = np.array([0, 9, 255])
shop_u_range = np.array([22, 38, 255])

building_l_range = np.array([96, 3, 9])
building_u_range = np.array([100, 255, 255])

trees_l_range = np.array([52, 28, 206])
trees_u_range = np.array([71, 255, 255])

def distance(pt_1, pt_2):
    pt_1 = np.array((pt_1[0], pt_1[1]))
    pt_2 = np.array((pt_2[0], pt_2[1]))
    return np.linalg.norm(pt_1-pt_2)

def closest_node(node, nodes):
    return nodes[cdist([node], nodes).argmin()]

    
def makecontours(image, lower_range, upper_range):
    font = cv2.FONT_HERSHEY_COMPLEX
    kernal = np.ones((5,5), np.uint8)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_range, upper_range)
    mask = cv2.erode(mask, kernal, iterations = 1)
    mask = cv2.dilate(mask, kernal, iterations = 2)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def checkcontours(image,midpoint, contours, y):
    for c in contours:
        approx = cv2.approxPolyDP(c,0.009 * cv2.arcLength(c, True), True)
        result = cv2.pointPolygonTest(c, midpoint, True)
        if result >= 0:
            all_pixel = []
            for a in range(0, len(c)):
                for b in range(0, len(c[a])):
                    all_pixel.append(c[a][b])

            n = approx.ravel() 
            anchr = closest_node(midpoint, all_pixel)
            anchr = (anchr[0], anchr[1])
            cv2.circle(image, anchr, radius=2, color=(0, 255, 0), thickness=-1)
            dist = distance(anchr, midpoint)
            #cv2.drawContours(image, [approx], 0, (0, 0, 255), 5)
            y = 1
            return anchr, dist, result, y
            break
        
        else:
            anchr = midpoint
            dist = 0
            y = 0
    return anchr, dist, result,y

        

    
def contourdist(image, midpoint, anchr):
    min_result = -100000
    flag = 0
    while True:
        for c in shop_contours:
            result = cv2.pointPolygonTest(c, midpoint, True)
            if result < 0:
                if result > min_result:
                    min_result = result
                    flag = 1

        for c in building_contours:
            result = cv2.pointPolygonTest(c, midpoint, True)
            if result < 0:
                if result > min_result:
                    min_result = result
                    flag = 2

        for c in road_contours:
            result = cv2.pointPolygonTest(c, midpoint, True)
            if result < 0:
                if result > min_result:
                    min_result = result
                    flag = 3
        return flag, min_result
        break

    

def checkflag(flag, min_result, midpoint, anchr):
    if flag == 1:
        for c in shop_contours:
            approx = cv2.approxPolyDP(c,0.009 * cv2.arcLength(c, True), True)
            result = cv2.pointPolygonTest(c, midpoint, True)
            if result == min_result:
                all_pixel = []
                for a in range(0, len(c)):
                    for b in range(0, len(c[a])):
                        all_pixel.append(c[a][b])

            
                pnt = closest_node(midpoint, all_pixel)
                pnt = (pnt[0], pnt[1])
                dist = distance(anchr, pnt )
                return dist
                break

    if flag == 2:
        for c in building_contours:
            approx = cv2.approxPolyDP(c,0.009 * cv2.arcLength(c, True), True)
            result = cv2.pointPolygonTest(c, midpoint, True)
            if result == min_result:
                all_pixel = []
                for a in range(0, len(c)):
                    for b in range(0, len(c[a])):
                        all_pixel.append(c[a][b])

            
                pnt = closest_node(midpoint, all_pixel)
                pnt = (pnt[0], pnt[1])
                dist = distance(anchr, pnt )
                return dist
                break

    if flag == 3:
        for c in road_contours:
            approx = cv2.approxPolyDP(c,0.009 * cv2.arcLength(c, True), True)
            result = cv2.pointPolygonTest(c, midpoint, True)
            if result == min_result:
                all_pixel = []
                for a in range(0, len(c)):
                    for b in range(0, len(c[a])):
                        all_pixel.append(c[a][b])

            
                pnt = closest_node(midpoint, all_pixel)
                pnt = (pnt[0], pnt[1])
                dist = distance(anchr, pnt )
                return dist
                break
    

In [23]:
def main():
    inp = input("Enter your image with file extension here:")
    x = inp.split(".")
    img_name = x[0]
    image = cv2.imread(inp)
    mdpnt = findmidpoint(image)
    global road_contours, shop_contours, building_contours, tree_contours
    road_contours = makecontours(image, road_l_range, road_u_range)
    shop_contours = makecontours(image, shop_l_range, shop_u_range)
    building_contours = makecontours(image, building_l_range, building_u_range)
    tree_contours = makecontours(image, trees_l_range, trees_u_range)
    y = 0
    anchor_side, dist, result, y  = checkcontours(image, mdpnt, shop_contours, y)
    
    if y == 0:
        anchor_side, dist, result, y  = checkcontours(image, mdpnt, building_contours, y)
    if y == 0:
        anchor_side = mdpnt
    
    flag , min_result = contourdist(image, mdpnt, anchor_side)
    length = checkflag(flag, min_result, mdpnt, anchor_side)
    if flag == 1:
        myflag = "Shop"
    elif flag ==2:
        myflag ="Building"
    else:
        myflag = "Road"
    
    length = int(length)

    print(img_name+","+str(length)+","+myflag)
    
    

In [24]:
main()

tmp_1,54,Building
